In [1]:
import os
import json
import re
from tqdm import tqdm
import pickle
import itertools
import networkx as nx
from matplotlib import pyplot as plt
import networkx

In [2]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

In [3]:
path = '../dtf_data/posts/'
files = sorted(os.listdir(path))

In [4]:
N = len(files)

In [5]:
N

472045

In [6]:
with open('companies_vc.txt', 'r') as f:
    companies_vc = f.read()

In [7]:
len(companies_vc.split('\n'))

476

In [8]:
companies_vc = companies_vc.split('\n')

In [9]:
with open('companies_habr.txt', 'r') as f:
    companies_habr = f.read()

In [10]:
len(companies_habr.split('\n'))

25244

In [11]:
companies_habr = companies_habr.split('\n')

In [12]:
with open('companies_dtf.txt', 'r') as f:
    companies_dtf = f.read()

In [13]:
len(companies_dtf.split('\n'))

69

In [14]:
companies_dtf = companies_dtf.split('\n')

In [15]:
companies = set(companies_vc).union(set(companies_habr).union(set(companies_dtf)))

In [17]:
data = 0
with open(path + files[63295]) as json_file:
    data = json.load(json_file)

In [18]:
files[236714]

'0000613250'

In [19]:
data['result'].keys()

dict_keys(['id', 'url', 'author', 'badges', 'commentsCount', 'commentsSeenCount', 'favoritesCount', 'cover', 'date', 'dateRFC', 'date_favorite', 'last_modification_date', 'hitsCount', 'intro', 'introInFeed', 'isEnabledComments', 'isEnabledLikes', 'isFavorited', 'isRepost', 'likes', 'subsite', 'similar', 'title', 'type', 'entryContent', 'commentatorsAvatars', 'webviewUrl', 'isPinned', 'canEdit', 'etcControls', 'highlight', 'blocks', 'subscribedToTreads', 'is_show_thanks', 'is_still_updating', 'is_filled_by_editors', 'isEditorial', 'audioUrl', 'hotness', 'commentEditor', 'summarize'])

In [20]:
data['result']['likes']

{'is_liked': 0, 'count': 172, 'summ': 172, 'is_hidden': False}

In [21]:
data['result']['intro']

'«Я разочарован, что мне пришлось покончить с игрой, которая была действительно важна для меня».'

In [22]:
data['result']['blocks']

[{'type': 'text',
  'data': {'text': '«Я разочарован, что мне пришлось покончить с игрой, которая была действительно важна для меня».',
   'text_truncated': '<<<same>>>'},
  'cover': True,
  'hidden': False,
  'anchor': ''},
 {'type': 'media',
  'data': {'items': [{'title': '',
     'author': '',
     'image': {'type': 'image',
      'data': {'uuid': 'c99a5f70-b0be-3d21-4b60-ff5d73378318',
       'width': 1620,
       'height': 801,
       'size': 3523982,
       'type': 'png',
       'color': '342920',
       'hash': '',
       'external_service': []}}}],
   'with_background': False,
   'with_border': False},
  'cover': True,
  'hidden': False,
  'anchor': ''},
 {'type': 'text',
  'data': {'text': '4 декабря глава Campo Santo, известной по Firewatch, [сообщил](https://dtf.ru/gameindustry/85418-valve-zamorozila-razrabotku-in-the-valley-of-gods-ot-avtorov-firewatch), что разработка In the Valley of Gods была заморожена — сотрудникам студии пришлось переключиться на другие проекты купивш

In [23]:
extracted_words = {}
for j in tqdm(range(N)):
#     print(f"Processing file:{files[j]}, number:{j}")
    if (j in (235865,235870)):
        continue
    if (files[j] in ('0000612178', '0000612230', '0000612173', '0000612215')):
        continue
    try:
        with open(path + files[j]) as json_file:
            data = json.load(json_file)
    except:
        continue
    if ('error' in data.keys()):
        continue
    num_blocks = len(data['result']['blocks'])
    extracted_words_set = set() 
    
    for i in range(num_blocks):
        if data['result']['blocks'][i]['type'] == 'text' or data['result']['blocks'][i]['type'] == 'header':
            extracted_words_set = extracted_words_set.union(set(re.findall(r'«[\w]+»', data['result']['blocks'][i]['data']['text'])))
            #print(re.findall(r'[\s]?[A-Z][A-Za-z]+[\s]|[\s]?[А-Я][А-Яа-я]+[\s]', data['result']['blocks'][i]['data']['text']))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][A-Za-z]*[\.][A-Z][A-Za-z]*|[А-Я][А-Яа-я]*[\.][А-Я][А-Яа-я]*', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z]+[^a-z]|[А-Я]+[^а-я]', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][a-z]+|[А-Я][а-я]+', data['result']['blocks'][i]['data']['text'])))

    extracted_words[files[j]] = [item.lower() for item in extracted_words_set]
    #        print(data['result']['blocks'][i]['data']['text'].split())
        

100%|█████████████████████████████████| 472045/472045 [06:18<00:00, 1247.59it/s]


In [18]:
extracted_words

{'0000000001': ['«маркетинг»',
  'sperasoft',
  'fdtf',
  'мероприятие',
  'ведьмак',
  'персоны',
  'как',
  '«геймдизайн»',
  'разные',
  'fhull',
  'я ',
  'толчком',
  'фадеевым',
  'павловым',
  'fwww',
  'мне',
  'дмитрием',
  'во',
  'давай',
  'коми',
  'отчасти',
  'получается',
  'игорем',
  'бабаев',
  'объяснил',
  'на',
  'леонидом',
  'fnekki',
  'писал',
  'fredirect',
  'бабаева',
  'когда',
  'пока',
  'хотя',
  'без',
  'fspecial',
  'максимом',
  'ещ',
  'концаренко',
  'нужно',
  'со',
  'водолазить',
  'аллодов',
  'леня',
  'давайте',
  'планов',
  'не',
  'владом',
  'destiny',
  'f%',
  'сиротиным',
  'friot',
  'о ',
  'пете',
  'пытался',
  'рынок',
  'кон',
  'fru',
  '«бизнес»',
  'рубрика',
  'unity',
  'сегодня',
  'занять',
  'титова',
  'канобу',
  'так',
  'продюсирование',
  'соцсети',
  'титовым',
  'черно',
  'сергея',
  'харитонову',
  'фоллаут',
  'двадцать',
  'наши',
  'думаю',
  'development',
  '«голый»',
  'fgames',
  'fplayraketa',
  'и ',
  

In [21]:
#with open('extracted_words.pkl', 'wb') as f:
#    pickle.dump(extracted_words, f)

In [24]:
'fl' in companies_vc

False

In [25]:
stop_words = set(['set', 
                  'далее', 
                  'развитие', 
                  'алгоритмы', 
                  'мария', 
                  'red', 
                  'main', 
                  'стартап', 
                  'бюро', 
                  'id', 
                  'поиск', 
                  'ясно',
                  'интерес',
                  'we',
                  'fl',
                  'too',
                  'успех',
                  'система'])

In [38]:
companies = set(companies_dtf)

In [26]:
companies.add('mail.ru')
companies.add('mail')
companies.add('mail.ru group')
companies.add('мэйл')
#companies.add('facebook')
#companies.add('meta')
companies = companies - stop_words

In [27]:
extracted_companies = {}
for key in extracted_words.keys():
    for word in extracted_words[key]:
        if word in companies:
            if not key in extracted_companies.keys():
                extracted_companies[key] = []
            extracted_companies[key].append(word)

In [28]:
len(extracted_companies)

121960

In [31]:
extracted_companies

{'0000000001': ['unity',
  'дарья',
  'добро',
  'raketa',
  'комитет',
  'dev',
  'spark',
  'nevosoft',
  'sperasoft',
  'макс',
  'xsolla',
  'playrix',
  'контент',
  'канобу'],
 '0000000004': ['климов', 'dev'],
 '0000000005': ['компания'],
 '0000000011': ['mail'],
 '0000000014': ['unity'],
 '0000000018': ['time'],
 '0000000019': ['zeptolab', 'top'],
 '0000000020': ['fplay'],
 '0000000027': ['valve'],
 '0000000028': ['life',
  'creative',
  'nival',
  'mail',
  'nevosoft',
  'real',
  'xsolla'],
 '0000000029': ['unity'],
 '0000000030': ['time'],
 '0000000034': ['mail'],
 '0000000037': ['zeptolab'],
 '0000000038': ['fibrum'],
 '0000000039': ['nival'],
 '0000000040': ['климов'],
 '0000000044': ['компания', 'plarium'],
 '0000000045': ['nevosoft', 'life', 'creative', 'real'],
 '0000000046': ['amazon', 'unity'],
 '0000000047': ['titan'],
 '0000000049': ['unity'],
 '0000000050': ['nival'],
 '0000000053': ['принц'],
 '0000000054': ['dev'],
 '0000000059': ['dev'],
 '0000000061': ['дарья'],

In [32]:
with open('extracted_companies_dtf.pkl', 'wb') as f:
   pickle.dump(extracted_companies, f)

In [29]:
edges = []
for key in extracted_companies.keys():
    if len(extracted_companies[key]) > 1:
        for element in itertools.product(*[extracted_companies[key], extracted_companies[key]]):
            edges.append(element)

In [30]:
len(edges)

1704232

In [27]:
edges_ = {}
for item in tqdm(edges):
    if item[0] != item[1]:
        if (not (item[0], item[1]) in edges_.keys()) and (not (item[1], item[0]) in edges_.keys()):
            edges_[item] = 0.0
        else:
            if (item[0], item[1]) in edges_.keys():
                edges_[item] += 1.0
            elif (item[1], item[0]) in edges_.keys():
                edges_[(item[1], item[0])] += 1.0

100%|█████████████████████████████| 1704232/1704232 [00:01<00:00, 964949.64it/s]


In [26]:
edges_

{('sperasoft', 'unity'): 27.0,
 ('sperasoft', 'канобу'): 19.0,
 ('sperasoft', 'raketa'): 19.0,
 ('sperasoft', 'playrix'): 27.0,
 ('sperasoft', 'добро'): 19.0,
 ('sperasoft', 'макс'): 19.0,
 ('sperasoft', 'dev'): 29.0,
 ('sperasoft', 'контент'): 23.0,
 ('sperasoft', 'spark'): 19.0,
 ('sperasoft', 'комитет'): 19.0,
 ('sperasoft', 'xsolla'): 19.0,
 ('sperasoft', 'nevosoft'): 21.0,
 ('sperasoft', 'дарья'): 19.0,
 ('unity', 'канобу'): 23.0,
 ('unity', 'raketa'): 19.0,
 ('unity', 'playrix'): 95.0,
 ('unity', 'добро'): 151.0,
 ('unity', 'макс'): 105.0,
 ('unity', 'dev'): 575.0,
 ('unity', 'контент'): 49.0,
 ('unity', 'spark'): 63.0,
 ('unity', 'комитет'): 19.0,
 ('unity', 'xsolla'): 33.0,
 ('unity', 'nevosoft'): 39.0,
 ('unity', 'дарья'): 23.0,
 ('канобу', 'raketa'): 19.0,
 ('канобу', 'playrix'): 19.0,
 ('канобу', 'добро'): 19.0,
 ('канобу', 'макс'): 21.0,
 ('канобу', 'dev'): 21.0,
 ('канобу', 'контент'): 27.0,
 ('канобу', 'spark'): 19.0,
 ('канобу', 'комитет'): 21.0,
 ('канобу', 'xsolla'): 2

In [28]:
G = nx.Graph()
for key in edges_.keys():
    G.add_weighted_edges_from([(key[0], key[1], edges_[key])])

In [29]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

In [30]:
number_to_adjust_by = 3
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [31]:
from networkx.algorithms import community
communities = community.greedy_modularity_communities(G, weight='weight', resolution=0.8)

In [32]:
# Create empty dictionaries
modularity_class = {}
modularity_color = {}
colors = (
'#3288bd',
'#66c2a5',
'#abdda4',
'#e6f598',
'#fee08b',
'#fdae61',
'#f46d43',
'#d53e4f',   
'#8c2d04',
)
#Loop through each community in the network
for community_number, community in enumerate(communities[:8]):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        modularity_color[name] = colors[community_number]

In [33]:
networkx.set_node_attributes(G, modularity_class, 'modularity_class')
networkx.set_node_attributes(G, modularity_color, 'modularity_color')

In [36]:
#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'
edge_color = 'gray'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Companies graph'


#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Companies", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
      #  ("Modularity Color", "$color[swatch]:modularity_color"),
]


#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              plot_width=2000, plot_height=2000,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)



#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.3, line_color=edge_color, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
#save(plot, filename=f"{title}.html")

In [273]:
with open('edges_vc.pkl', 'wb') as f:
    pickle.dump(edges, f)

In [274]:
with open('edges_vc_.pkl', 'wb') as f:
    pickle.dump(edges_, f)

In [72]:
s

In [34]:
len(G.nodes)

1928

In [35]:
len(G.edges)

81666

In [33]:
#for Eldar save
networkx.set_node_attributes(G, name='size', values=adjusted_node_size)

In [34]:
networkx.set_node_attributes(G, modularity_color, 'color')

In [35]:
networkx.write_gml(G, "dtf_posts.gml")